## Flight Price Prediction(EDA + Feature Engineering)

1. Data Gathering
2. Data Assessing
3. Data Data Cleaning
4. Data Preprocessing 
5. Model Exploration/Selection
6. Model Training
7. Save Model


# 1. Data Gathering

#### Dataset: https://github.com/krishnaik06/5-Days-Live-EDA-and-Feature-Engineering/tree/main/Flight%20Prediction

In [3]:
#importing basics libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


# Modelling-
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

import pickle

In [18]:
df = pd.read_excel('Data_Train.xlsx')
df1 = pd.read_excel('Test_set.xlsx')

# 2. Data Assessment

In [5]:
df.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


In [19]:
df1.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [21]:
print('The shape of the tain data is:', df.shape)
print('The shape of the test data is:', df1.shape)

The shape of the tain data is: (10683, 11)
The shape of the test data is: (2671, 10)


In [7]:
df.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

In [22]:
df1.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')

In [8]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [10]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [23]:
df1.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

Drop the null values

In [12]:
print('Number of duplicated rows :', df.duplicated().sum())


Number of duplicated rows : 220


In [ ]:
df[df.duplicated()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
683,Jet Airways,1/06/2019,Delhi,Cochin,DEL → NAG → BOM → COK,14:35,04:25 02 Jun,13h 50m,2 stops,No info,13376
1061,Air India,21/05/2019,Delhi,Cochin,DEL → GOI → BOM → COK,22:00,19:15 22 May,21h 15m,2 stops,No info,10231
1348,Air India,18/05/2019,Delhi,Cochin,DEL → HYD → BOM → COK,17:15,19:15 19 May,26h,2 stops,No info,12392
1418,Jet Airways,6/06/2019,Delhi,Cochin,DEL → JAI → BOM → COK,05:30,04:25 07 Jun,22h 55m,2 stops,In-flight meal not included,10368
1674,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,18:25,21:20,2h 55m,non-stop,No info,7303
...,...,...,...,...,...,...,...,...,...,...,...
10594,Jet Airways,27/06/2019,Delhi,Cochin,DEL → AMD → BOM → COK,23:05,12:35 28 Jun,13h 30m,2 stops,No info,12819
10616,Jet Airways,1/06/2019,Delhi,Cochin,DEL → JAI → BOM → COK,09:40,12:35 02 Jun,26h 55m,2 stops,No info,13014
10634,Jet Airways,6/06/2019,Delhi,Cochin,DEL → JAI → BOM → COK,09:40,12:35 07 Jun,26h 55m,2 stops,In-flight meal not included,11733
10672,Jet Airways,27/06/2019,Delhi,Cochin,DEL → AMD → BOM → COK,23:05,19:00 28 Jun,19h 55m,2 stops,In-flight meal not included,11150


There is no need to drop the duplicate rows because they are not entirely duplicate values

# Data Preprocessing 
### Feature engineering

In [13]:
# Handle missing values for 'Route' and 'Total_Stops' in train_df 
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [15]:
# Feature Engineering
def preprocess_data(data_df):
    # Extract day, month, and weekday from 'Date_of_Journey'
    data_df['Journey_Day'] = pd.to_datetime(data_df['Date_of_Journey'], format='%d/%m/%Y').dt.day.astype(int)
    data_df['Journey_Month'] = pd.to_datetime(data_df['Date_of_Journey'], format='%d/%m/%Y').dt.month.astype(int)
    data_df['Journey_Weekday'] = pd.to_datetime(data_df['Date_of_Journey'], format='%d/%m/%Y').dt.weekday.astype(int)

    # Calculate the duration in minutes from 'Duration'
    def duration_to_minutes(duration):
        duration_list = duration.split()
        if len(duration_list) == 2:
            return int(duration_list[0][:-1]) * 60 + int(duration_list[1][:-1])
        elif 'h' in duration:
            return int(duration_list[0][:-1]) * 60
        else:
            return int(duration_list[0][:-1])

    data_df['Duration_minutes'] = data_df['Duration'].apply(duration_to_minutes).astype(int)

    # Convert 'Dep_Time' and 'Arrival_Time' to datetime format and extract hour and minute
    data_df['Dep_Time'] = pd.to_datetime(data_df['Dep_Time'])
    data_df['Dep_Hour'] = data_df['Dep_Time'].dt.hour.astype(int)
    data_df['Dep_Minute'] = data_df['Dep_Time'].dt.minute.astype(int)

    data_df['Arrival_Time'] = pd.to_datetime(data_df['Arrival_Time'])
    data_df['Arrival_Hour'] = data_df['Arrival_Time'].dt.hour.astype(int)
    data_df['Arrival_Minute'] = data_df['Arrival_Time'].dt.minute.astype(int)

    return data_df


# Preprocess combined_df
combined_df = preprocess_data(df)


C:\Users\HP\AppData\Local\Temp\ipykernel_11336\546624599.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_df['Dep_Time'] = pd.to_datetime(data_df['Dep_Time'])
C:\Users\HP\AppData\Local\Temp\ipykernel_11336\546624599.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_df['Arrival_Time'] = pd.to_datetime(data_df['Arrival_Time'])


In [24]:
combined_df1 = preprocess_data(df1)

C:\Users\HP\AppData\Local\Temp\ipykernel_11336\546624599.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_df['Dep_Time'] = pd.to_datetime(data_df['Dep_Time'])
C:\Users\HP\AppData\Local\Temp\ipykernel_11336\546624599.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_df['Arrival_Time'] = pd.to_datetime(data_df['Arrival_Time'])


In [16]:
df.head(2)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Weekday,Duration_minutes,Dep_Hour,Dep_Minute,Arrival_Hour,Arrival_Minute
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,2023-08-14 22:20:00,2023-03-22 01:10:00,2h 50m,non-stop,No info,3897,24,3,6,170,22,20,1,10
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,2023-08-14 05:50:00,2023-08-14 13:15:00,7h 25m,2 stops,No info,7662,1,5,2,445,5,50,13,15


In [25]:
df1.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Journey_Day,Journey_Month,Journey_Weekday,Duration_minutes,Dep_Hour,Dep_Minute,Arrival_Hour,Arrival_Minute
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,2023-08-14 17:30:00,2023-06-07 04:25:00,10h 55m,1 stop,No info,6,6,3,655,17,30,4,25
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,2023-08-14 06:20:00,2023-08-14 10:20:00,4h,1 stop,No info,12,5,6,240,6,20,10,20
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,2023-08-14 19:15:00,2023-05-22 19:00:00,23h 45m,1 stop,In-flight meal not included,21,5,1,1425,19,15,19,0
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,2023-08-14 08:00:00,2023-08-14 21:00:00,13h,1 stop,No info,21,5,1,780,8,0,21,0
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,2023-08-14 23:55:00,2023-06-25 02:45:00,2h 50m,non-stop,No info,24,6,0,170,23,55,2,45


In [17]:
# Drop the following columns 'Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration'
df.drop(['Date_of_Journey', 'Dep_Time', 'Arrival_Time', 'Duration', 'Route'], axis = 1, inplace = True)

In [ ]:
# Display the modified train_df
df.head(2)

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Journey_Weekday,Duration_minutes,Dep_Hour,Dep_Minute,Arrival_Hour,Arrival_Minute
0,IndiGo,Banglore,New Delhi,non-stop,No info,3897,24,3,6,170,22,20,1,10
1,Air India,Kolkata,Banglore,2 stops,No info,7662,1,5,2,445,5,50,13,15


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10682 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Airline           10682 non-null  object
 1   Source            10682 non-null  object
 2   Destination       10682 non-null  object
 3   Total_Stops       10682 non-null  object
 4   Additional_Info   10682 non-null  object
 5   Price             10682 non-null  int64 
 6   Journey_Day       10682 non-null  int32 
 7   Journey_Month     10682 non-null  int32 
 8   Journey_Weekday   10682 non-null  int32 
 9   Duration_minutes  10682 non-null  int32 
 10  Dep_Hour          10682 non-null  int32 
 11  Dep_Minute        10682 non-null  int32 
 12  Arrival_Hour      10682 non-null  int32 
 13  Arrival_Minute    10682 non-null  int32 
dtypes: int32(8), int64(1), object(5)
memory usage: 918.0+ KB


## Feature Encoding 

In [ ]:
# define numerical & categorical columns in train data
numeric_features = [feature for feature in combined_df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in combined_df.columns if df[feature].dtype == 'O']

# print numerical & categorical columns in train data
print('We have {} numerical features in train data and they as as follows : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features train data and they as as follows: {}'.format(len(categorical_features), categorical_features))

We have 9 numerical features in train data and they as as follows : ['Price', 'Journey_Day', 'Journey_Month', 'Journey_Weekday', 'Duration_minutes', 'Dep_Hour', 'Dep_Minute', 'Arrival_Hour', 'Arrival_Minute']

We have 5 categorical features train data and they as as follows: ['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info']


In [ ]:
print('Number of unique data points in categorical features in Train data')
print('Number of unique data points in Airline:', df['Airline'].unique())
print('Number of unique data points in Source:',df['Source'].unique())
print('Number of unique data points in Destination:',df['Destination'].unique())
print('Number of unique data points in Total_Stops:', df['Total_Stops'].unique())
print('Number of unique data points in Additional_Info:', df['Additional_Info'].unique())

Number of unique data points in categorical features in Train data
Number of unique data points in Airline: ['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']
Number of unique data points in Source: ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']
Number of unique data points in Destination: ['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad']
Number of unique data points in Total_Stops: ['non-stop' '2 stops' '1 stop' '3 stops' '4 stops']
Number of unique data points in Additional_Info: ['No info' 'In-flight meal not included' 'No check-in baggage included'
 '1 Short layover' 'No Info' '1 Long layover' 'Change airports'
 'Business class' 'Red-eye flight' '2 Long layover']


In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

# Define lists of categorical and numerical features for combined_df
categorical_features = ['Airline', 'Source', 'Destination', 'Total_Stops', 'Additional_Info']
numerical_features = ['Price', 'Journey_Day', 'Journey_Month', 'Journey_Weekday', 'Duration_minutes', 'Dep_Hour', 'Dep_Minute', 'Arrival_Hour', 'Arrival_Minute']

# Separate numerical and categorical features in combined_df
numerical_df = df[numerical_features]
categorical_df = df[categorical_features]

# Apply one-hot encoding to categorical features
encoder = OneHotEncoder(drop='first')
encoded_categorical = encoder.fit_transform(categorical_df)

# Apply standard scaling to numerical features
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(numerical_df)

# Convert encoded_categorical to a DataFrame with appropriate column names
encoded_categorical_df = pd.DataFrame(encoded_categorical.toarray(), columns=encoder.get_feature_names_out(categorical_features))

# Combine numerical and encoded categorical features
df = pd.concat([pd.DataFrame(scaled_numerical, columns=numerical_features), encoded_categorical_df], axis=1)

In [ ]:
df.head()

,Price,Journey_Day,Journey_Month,Journey_Weekday,Duration_minutes,Dep_Hour,Dep_Minute,Arrival_Hour,Arrival_Minute,Airline_Air India,...,Total_Stops_non-stop,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,-1.125535,1.237288,-1.467402,1.527476,-0.931498,1.654154,-0.234950,-1.800436,-0.890014,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.309068,-1.475307,0.250289,-0.466048,-0.389953,-1.303113,1.363607,-0.050909,-0.587094,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.039783,-0.531796,1.109135,1.527476,0.978679,-0.607286,0.031476,-1.363054,0.018745,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-0.622209,-0.177979,0.250289,1.527476,-0.626263,0.958326,-1.034229,1.407030,0.321664,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.914006,-1.475307,-1.467402,0.530714,-0.705034,0.610412,1.363607,1.115442,0.624584,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Model Explortion/Selection

In [ ]:
X = df.drop(['Price'], axis = 1)
y = df['Price']

In [ ]:
# Splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [ ]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 0.5397
- Mean Absolute Error: 0.3759
- R2 Score: 0.7065
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 80245474.5735
- Mean Absolute Error: 1735874.7381
- R2 Score: -6247536736514238.0000


Lasso
Model performance for Training set
- Root Mean Squared Error: 0.9961
- Mean Absolute Error: 0.7930
- R2 Score: 0.0000
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1.0156
- Mean Absolute Error: 0.7899
- R2 Score: -0.0007


Ridge
Model performance for Training set
- Root Mean Squared Error: 0.5413
- Mean Absolute Error: 0.3772
- R2 Score: 0.7047
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 0.5846
- Mean Absolute Error: 0.3775
- R2 Score: 0.6684


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 0.3661
- Mean Absolute Error: 0.1946
- R2 Score: 